### For given aoi, prepare TCI and NDVI sentinel images 

In [1]:
import os
import geopandas as gp
import numpy as np
import rasterio
import re
import tempfile
import pyproj
import uuid
import rasterio.mask

from rasterio.windows import Window
from rasterio.plot import reshape_as_raster
from rasterio import Affine
from rasterio.merge import merge
from rasterio.warp import calculate_default_transform, reproject, Resampling
from shapely.geometry import Polygon, box
from shapely.ops import transform
from pathlib import Path
from datetime import datetime, timedelta
from sentinel2download.downloader import Sentinel2Downloader
from sentinel2download.overlap import Sentinel2Overlap

In [2]:
def crop(input_path, output_path, polygon, name, date):
    with rasterio.open(input_path) as src:
        out_image, out_transform = rasterio.mask.mask(src, [polygon], crop=True)
        # print(out_transform)
        out_meta = src.meta
        
        out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform,
                 "nodata": 0,
                        })

    with rasterio.open(output_path, "w", **out_meta) as dest:
        dest.update_tags(name=name, start_date=date, end_date=date)
        dest.write(out_image)

In [3]:
def convert(input_path, output_path, driver='GTiff'):
    with rasterio.open(input_path) as src:
        raster = src.read()
        crs = str(src.crs)
        
        # print("CONVERTED CRS")
        # print(crs)
                
        kwargs = src.meta.copy()
        kwargs.update({'driver': driver})
        
        
        with rasterio.open(output_path, 'w', **kwargs) as dst:
            dst.write(raster)
        return crs

In [4]:
def to_crs(poly, target, current='EPSG:4326'):
    # print(f"TARGET CRS: {target}")
    project = pyproj.Transformer.from_crs(pyproj.CRS(current), pyproj.CRS(target), always_xy=True).transform
    transformed_poly = transform(project, poly)
    return transformed_poly 

In [5]:
def generate_dates(delta=20, format='%Y-%m-%d'):
    now = datetime.now()
    end_date = datetime(now.year, now.month, now.day)

    while True:
        start_date = end_date - timedelta(days=delta)
        yield datetime.strftime(start_date, format), datetime.strftime(end_date, format)
        end_date = start_date

In [6]:
def load_images(aoi_path, api_key, output_dir):
    
    overlap = Sentinel2Overlap(aoi_path)
    tiles = overlap.overlap()
    print(f"Overlap tiles: {tiles}")
    
    loader = Sentinel2Downloader(api_key)
    loadings = dict()
    for tile in tiles:
        print(f"Loading images for tile: {tile}...")
        for start_date, end_date in generate_dates():
            print(f"Dates from {start_date} to {end_date}")
            loaded = loader.download('L2A',
                                     [tile],
                                     start_date=start_date,
                                     end_date=end_date,
                                     output_dir=output_dir,                       
                                     bands=BANDS,
                                     constraints=CONSTRAINTS)

            if loaded:
                print(f"Loading images for tile {tile} finished")
                loadings[tile] = loaded
                break
            else:
                print("No matching images")
    return loadings

In [7]:
def last_image_paths(loaded):
    date_pattern = r"_(\d+)T\d+_"
    dates = list()
    for path, _ in loaded:
        if path.endswith('.jp2'):
            search = re.search(date_pattern, path)
            date = search.group(1)
            date = datetime.strptime(date, '%Y%m%d')
            dates.append(date)
    
    last_date = max(dates)
    last_date = datetime.strftime(date, '%Y%m%d')
    paths = list()
    for path, _ in loaded:
        if path.endswith('.jp2') and last_date in path:
            paths.append(path)
    return paths, last_date

In [8]:
def filter_loadings(loadings):
    filtered = dict()
    for tile, image_paths in loadings.items():        
        paths, date = last_image_paths(image_paths)
        bands_paths = dict()
        for path in paths:
            if 'B04' in path:
                bands_paths['RED'] = path
            if 'B08' in path:
                bands_paths['NIR'] = path
            if 'TCI' in path:
                bands_paths['TCI'] = path
        filtered[tile] = dict(paths=bands_paths, date=date)
    return filtered

In [9]:
def _save_original_ndvi(ndvi, src, path, nodata=-1000):
    # mask nan values
    nan_mask = np.isnan(ndvi)
    ndvi[nan_mask] = nodata
    
    # Set spatial characteristics of the output object
    kwargs = src.meta.copy()    
    kwargs.update(
        dtype=rasterio.float32,
        driver='GTiff',
        nodata=nodata,
        count = 1)
    
    # print(f"Metadata of NDVI: {kwargs}")

    # Create the file
    with rasterio.open(path, 'w', **kwargs) as dst:
         dst.write(ndvi.astype(rasterio.float32), 1)

In [10]:
def scale(ndvi, a=1, b=255, nodata=0.0):
    # ndvi is in range [-1; 1], nodata is setted to 0.0 value. Be careful with comprassions!
    min = -1 # np.nanmin(ndvi)
    max = 1 # np.nanmax(ndvi)
    scaled = (b - a) * (ndvi - min) / (max - min) + a
    scaled = np.around(scaled)
    scaled[np.isnan(scaled) == True] = nodata
    scaled = scaled.astype(np.uint8)
    return scaled

In [11]:
def prepare_colors(colors):
    colors = np.load(COLORMAP_BRBG)
    # delete last channel, we use rgb
    colors = np.delete(colors, 3, axis=1)
    # colormap colors values in range [0-255], but in our case 0 - no data, -> have to color as [0, 0, 0] 
    colors[colors == 0] = 1
    colors[0] = [0, 0, 0]
    return colors

In [12]:
def color_ndvi(scaled, colors):
    colored = np.reshape(colors[scaled.flatten()], tuple((*scaled.shape, 3)))
    colored = reshape_as_raster(colored)
    return colored

In [13]:
def NDVI(nir_path, red_path, save_path):
    print("Calculating NDVI...")
    # Allow division by zero
    np.seterr(divide='ignore', invalid='ignore')
    
    with rasterio.open(nir_path) as src:
        nir = src.read(1).astype(rasterio.float32)
        crs = str(src.crs)
    with rasterio.open(red_path) as src:
        red = src.read(1).astype(rasterio.float32)

    # Calculate NDVI
    ndvi = ((nir - red) / (nir + red)) 
    
    scaled = scale(ndvi)
    colored = color_ndvi(scaled, COLORS) 
    
    
    # Set spatial characteristics of the output object
    kwargs = src.meta.copy()    
    kwargs.update(
        dtype=rasterio.uint8,
        driver='GTiff',
        nodata=0,
        count=3, )
    
    # print(f"Metadata of NDVI: {kwargs}")

    # Create the file
    with rasterio.open(save_path, 'w', **kwargs) as dst:
         dst.write(colored)
    print("NDVI calculation finished")
    return crs

In [14]:
def get_path(path, layer, base, name, suffix=None):
    if layer == 'NDVI':
        stem = Path(path).stem
        stem = re.sub(r"_B\d{2}", '_NDVI', stem)
    else:
        stem = Path(path).stem
    path = os.path.join(base, f"{name}_{stem}.tif")
    if suffix:
         path += suffix
    os.makedirs(os.path.dirname(path), exist_ok=True)
    return path

In [15]:
def reproject_image(input_path, dst_crs='EPSG:3857'):
    with rasterio.open(input_path) as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
            })
        output_path = f"{input_path[:-4]}_{dst_crs.replace(':', '_').lower()}.tif" 
        with rasterio.open(output_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)
    return output_path 

In [16]:
def get_start_end_date(filtered):
    dates = list()
    for tile, properties in filtered.items():
        date = properties['date']
        date = datetime.strptime(date, '%Y%m%d')
        dates.append(date)
    
    start_date = max(dates)
    end_date = min(dates)
    return datetime.strftime(start_date, '%Y-%m-%d'), datetime.strftime(end_date, '%Y-%m-%d')

In [17]:
def generate_mosaic(layer, files_to_mosaic, name, start_date, end_date, results_dir):
    src_files_to_mosaic = list()
    for fp in files_to_mosaic:
        src = rasterio.open(fp)
        src_files_to_mosaic.append(src)
    
    # crs of first input will be used
    mosaic, out_trans = merge(src_files_to_mosaic)
    # print(src)
    out_meta = src.meta.copy()
    # print(out_meta)
    # print(out_meta['crs'])

    # Update the metadata
    out_meta.update({"driver": "GTiff",
                     "height": mosaic.shape[1],
                     "width": mosaic.shape[2],
                     "transform": out_trans,
                     })
    
    out_fp = os.path.join(results_dir, f"{name}_{layer}_mosaic.tif.temp") 
    # print(out_fp)
    os.makedirs(os.path.dirname(out_fp), exist_ok=True)
    with rasterio.open(out_fp, "w", **out_meta) as dest:
        dest.update_tags(name=name, start_date=start_date, end_date=end_date)
        dest.write(mosaic)
    os.rename(out_fp, out_fp[:-5])

In [18]:
def calculate(layer, temp_dir, bbox, name, properties):
    """
    Preparing true color Sentinel2 and NDVI images for given AOI
    """
    print("Calculating TCI and NDVI for given AOI...")
    
    image_paths = properties['paths']
    date = properties['date']
  
    if layer == 'TCI':       
        # convert .jp2 image to .tif
        path = image_paths['TCI']
        temp_path = get_path(path, layer, temp_dir, name)    
        print(f"Save {layer} image path: {temp_path}")
            
        target_crs = convert(path , temp_path) 
            
    if layer == 'NDVI':
        path = image_paths['RED']
        temp_path = get_path(path, layer, temp_dir, name)    
        print(f"Save {layer} image path: {temp_path}")
            
        # calculate NDVI
        target_crs = NDVI(image_paths['NIR'], image_paths['RED'], temp_path)
    
    # print("TARGET CRS")
    # print(target_crs)
    bound_box = to_crs(bbox, target_crs)   

    
    temp_output_path = get_path(path, layer, temp_dir, name, suffix=".temp")
    print(f"Temporary {layer} output path: {temp_output_path}")
    
    # crop .tif image by aoi
    crop(temp_path, temp_output_path, bound_box, name, date)   
    
    
    output_path = temp_output_path[:-5]
    print(f"Rename temp file: {temp_output_path} to {output_path}")
    os.rename(temp_output_path, output_path)
    
    
    # for warping in mosaic, reproject to same crs, set to 'EPSG:3857' 
    output_path = reproject_image(output_path)  
    
    print(output_path)
        
    
    return output_path

### Execution part

In [19]:
BASE = f"/home/{os.getenv('NB_USER')}/work"

In [20]:
# os.environ['AOI'] = os.path.join(BASE, "notebooks/sip/Kharkiv.geojson") # Pechenihy.geojson

aoi_path = os.getenv('AOI') 
if not aoi_path:
    raise RuntimeError("Add AOI env var for calculations")
    
aoi_path

'/home/jovyan/work/notebooks/sip/Pechenihy.geojson'

In [21]:
API_KEY = os.path.join(BASE, ".secret/sentinel2_google_api_key.json")
LOAD_DIR = os.path.join(BASE, "satellite_imagery")
RESULTS_DIR = os.path.join(BASE, "results/sip")

SIP_DIR = os.path.join(BASE, "notebooks/sip")
COLORMAP_BRBG = os.path.join(SIP_DIR, "magma.npy") 
COLORS = prepare_colors(COLORMAP_BRBG)


BANDS = {'TCI', 'B04', 'B08', }
CONSTRAINTS = {'NODATA_PIXEL_PERCENTAGE': 10.0, 'CLOUDY_PIXEL_PERCENTAGE': 5.0, }

LAYERS = ['TCI', 'NDVI', ]

#### Get bound box

In [22]:
aoi = gp.read_file(aoi_path)    
bbox = box(*aoi.total_bounds)

NAME = Path(aoi_path).stem

#### Loading images

In [23]:
loadings = load_images(aoi_path, API_KEY, LOAD_DIR)
filtered = filter_loadings(loadings)

Overlap tiles: ['36UYA']
Loading images for tile: 36UYA...
Dates from 2020-11-14 to 2020-12-04
No matching images
Dates from 2020-10-25 to 2020-11-14
No matching images
Dates from 2020-10-05 to 2020-10-25
No matching images
Dates from 2020-09-15 to 2020-10-05
Loading images for tile 36UYA finished


In [24]:
filtered

{'36UYA': {'paths': {'NIR': '/home/jovyan/work/satellite_imagery/S2B_MSIL2A_20200923T083659_N0214_R064_T36UYA_20200925T161337/T36UYA_20200923T083659_B08_10m.jp2',
   'RED': '/home/jovyan/work/satellite_imagery/S2B_MSIL2A_20200923T083659_N0214_R064_T36UYA_20200925T161337/T36UYA_20200923T083659_B04_10m.jp2',
   'TCI': '/home/jovyan/work/satellite_imagery/S2B_MSIL2A_20200923T083659_N0214_R064_T36UYA_20200925T161337/T36UYA_20200923T083659_TCI_10m.jp2'},
  'date': '20200923'}}

### TCI, NDVI and mosaic calculation

In [25]:
SIP_DIR

'/home/jovyan/work/notebooks/sip'

In [26]:
RESULTS_DIR

'/home/jovyan/work/results/sip'

In [27]:
# if true, mosaic will be created, else every file will be saved (for debugging)
ONLY_MOSAIC = True

In [28]:
with tempfile.TemporaryDirectory(dir=SIP_DIR) as tmpdirname:
    layer_paths = dict()
    if not ONLY_MOSAIC:
        tmpdirname = os.path.join(SIP_DIR, "results")
        
    print(f"Сreated temporary directory for calculations: {tmpdirname}")   
    for layer in LAYERS:
        print(f"\nCalculating {layer}:\n")
        paths = list()
        for tile, properties in filtered.items():
            print(f"\nExecution for tile {tile}")
            
            save_path = calculate(layer, tmpdirname, bbox, NAME, properties)
            
            paths.append(save_path)
            
        layer_paths[layer] = paths
        print(f"Execution for tile {tile} finished")
    
    # print(layer_paths)
    
    print("\nCREATING MOSAIC\n")
    start_date, end_date = get_start_end_date(filtered)
    print(f"Date range for mosaic from {start_date} till {end_date}")

    for layer, files_to_mosaic in layer_paths.items():
        print(f"\nGenerating mosaic for {layer}")
        generate_mosaic(layer, files_to_mosaic, NAME, start_date, end_date, RESULTS_DIR)
        print("\nGenerating mosaic finished")

Сreated temporary directory for calculations: /home/jovyan/work/notebooks/sip/results

Calculating TCI:


Execution for tile 36UYA
Calculating TCI and NDVI for given AOI...
Save TCI image path: /home/jovyan/work/notebooks/sip/results/Pechenihy_T36UYA_20200923T083659_TCI_10m.tif
Temporary TCI output path: /home/jovyan/work/notebooks/sip/results/Pechenihy_T36UYA_20200923T083659_TCI_10m.tif.temp
Rename temp file: /home/jovyan/work/notebooks/sip/results/Pechenihy_T36UYA_20200923T083659_TCI_10m.tif.temp to /home/jovyan/work/notebooks/sip/results/Pechenihy_T36UYA_20200923T083659_TCI_10m.tif
/home/jovyan/work/notebooks/sip/results/Pechenihy_T36UYA_20200923T083659_TCI_10m_epsg_3857.tif
Execution for tile 36UYA finished

Calculating NDVI:


Execution for tile 36UYA
Calculating TCI and NDVI for given AOI...
Save NDVI image path: /home/jovyan/work/notebooks/sip/results/Pechenihy_T36UYA_20200923T083659_NDVI_10m.tif
Calculating NDVI...
NDVI calculation finished
Temporary NDVI output path: /home/jov